# Introduction

This exercise shows how to download RBMC observation files in RINEX 3, compile and run the utilities needed to transform it to RINEX 2.11.

This notebook uses crx2rnx,cnvrnx2-rnx2-v3  and teqc to perform the transformations.

Change the URL bellow to use a different file.

In [ ]:
link="""https://geoftp.ibge.gov.br/informacoes_sobre_posicionamento_geodesico/rbmc/dados_RINEX3/2021/236/RJNI00BRA_R_20212360000_01D_15S_MO.crx.gz"""
rbmcfile=link.split("/")[-1]

# cnvrnx3

## Downloading source code 

In [ ]:
!wget "http://acc.igs.org/misc/cnvrnx3-rnx2-v3.f"
fname='cnvrnx3-rnx2-v3.f'

--2021-11-01 19:42:40--  http://acc.igs.org/misc/cnvrnx3-rnx2-v3.f
Resolving acc.igs.org (acc.igs.org)... 52.29.216.227
Connecting to acc.igs.org (acc.igs.org)|52.29.216.227|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 71023 (69K) [text/plain]
Saving to: ‘cnvrnx3-rnx2-v3.f.1’

cnvrnx3-rnx2-v3.f.1 100%[===================>]  69.36K   324KB/s    in 0.2s    

2021-11-01 19:42:41 (324 KB/s) - ‘cnvrnx3-rnx2-v3.f.1’ saved [71023/71023]



## fixing the code for unix compilation, as the source instructed

In [ ]:
fcode=open(fname,'r')
linuxCode=fcode.read()
linuxCode=linuxCode.replace(
"""C WINDOWS (Intel Visual Fortran)          ! Comment or uncomment
      call getarg(1,rnxfilev3,istatus1)   ! the following statements
      call getarg(2,cicreate,istatus2)    ! to switch from WINDOWS to UNIX/LINUX
      call getarg(3,gr,istatus3)
      slash = '\\'
    5 continue
      if(istatus1.gt.100.or.istatus2.ne.2.
     .or.istatus3.ne.1.or.ibad.eq.1) then""",
"""C WINDOWS (Intel Visual Fortran)          ! Comment or uncomment
c      call getarg(1,rnxfilev3,istatus1)   ! the following statements
c      call getarg(2,cicreate,istatus2)    ! to switch from WINDOWS to UNIX/LINUX
c      call getarg(3,gr,istatus3)
c      slash = '\\'
c    5 continue
c      if(istatus1.gt.100.or.istatus2.ne.2.
c     .or.istatus3.ne.1.or.ibad.eq.1) then""")
linuxCode=linuxCode.replace("""c     call getarg(1,rnxfilev3)
c     call getarg(2,cicreate)
c     call getarg(3,gr)
c     slash = '/'
c   5 continue
c     if(ibad.eq.1) then""",    
"""      call getarg(1,rnxfilev3)
      call getarg(2,cicreate)
      call getarg(3,gr)
      slash = '/'
    5 continue
      if(ibad.eq.1) then"""
)
fLinuxCode=open('tempcode.f','w')
fLinuxCode.write(linuxCode)
fLinuxCode.close()


## Compiling

In [ ]:
!gfortran tempcode.f -o cnvrnx3-rnx2-v3 

# crx2rnx

In [ ]:
!wget https://raw.githubusercontent.com/geospace-code/georinex/main/src/georinex/rnxcmp/source/crx2rnx.c
!gcc crx2rnx.c -o crx2rnx

--2021-11-01 19:42:41--  https://raw.githubusercontent.com/geospace-code/georinex/main/src/georinex/rnxcmp/source/crx2rnx.c
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36738 (36K) [text/plain]
Saving to: ‘crx2rnx.c.1’

crx2rnx.c.1         100%[===================>]  35.88K  --.-KB/s    in 0.003s  

2021-11-01 19:42:42 (12.0 MB/s) - ‘crx2rnx.c.1’ saved [36738/36738]



# Running

## Downloading RINEX 3 file from RBMC

In [ ]:
!wget "$link"

--2021-11-01 19:42:42--  https://geoftp.ibge.gov.br/informacoes_sobre_posicionamento_geodesico/rbmc/dados_RINEX3/2021/236/RJNI00BRA_R_20212360000_01D_15S_MO.crx.gz
Resolving geoftp.ibge.gov.br (geoftp.ibge.gov.br)... 191.233.136.86
Connecting to geoftp.ibge.gov.br (geoftp.ibge.gov.br)|191.233.136.86|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3558674 (3.4M) [application/x-gzip]
Saving to: ‘RJNI00BRA_R_20212360000_01D_15S_MO.crx.gz’

RJNI00BRA_R_2021236 100%[===================>]   3.39M  2.94MB/s    in 1.2s    

2021-11-01 19:42:44 (2.94 MB/s) - ‘RJNI00BRA_R_20212360000_01D_15S_MO.crx.gz’ saved [3558674/3558674]



## Unzipping and transforming from hatanaka to rnx

In [ ]:
!gunzip -f $rbmcfile
crxfile=rbmcfile[:-3]
!./crx2rnx $crxfile

The file RJNI00BRA_R_20212360000_01D_15S_MO.rnx already exists. Overwrite?(n)y


In [ ]:
#Check the file sizes
!ls -la RJ*

-rw-r--r-- 1 root root  9394156 Aug 25 23:01 RJNI00BRA_R_20212360000_01D_15S_MO.crx
-rw-r--r-- 1 root root 32487683 Nov  1 19:43 RJNI00BRA_R_20212360000_01D_15S_MO.rnx
-rw-r--r-- 1 root root 17187579 Nov  1 19:28 RJNI00BRA_R_20212360000_01D_15S_MO.rnx.gps+glo.rnx2


## Converting to rinex 3

In [ ]:
#This is only to show how the command works
!./cnvrnx3-rnx2-v3

 
 Program "CNVRNX3-RNX2" V3.1.3
 for converting RINEX 3 to RINEX 2
 Copyright (C) 2012-2014 Oscar L. Colombo.
 
  
  ERROR in line command "cnvrnx3-rnx2-v3"
  The correct command is: 
  
 "cnvrnx3-rnx2-v3  rnxv3 i o"
  
  "rnxv3",   = Name of the RINEX file to be converted -- up to 100 characters long.
  
  "i"    is a  2-character flag.
  "o"    is a  1-character flag.
  Make  o = g  to convert only the Rinex3 GPS data;
          = m  to convert Rinex 3 GPS plus GLONASS data.
  The GPS data are copied with the "i" options below:
  (Glonass data are copied if their types correspond 
  to those of GPS chosen according to the "i" options.)
  Because GLONASS' data are assigned places based on
  those assigned to GPS', this program does not create 
  GLONASS data-only Rinex 2 files.
  
  To copy L2 and the phase of L2C if both present:
   i = +0  (and only the RINEX2 file is created) 
  NOT to copy any L2C data, even if they are present:
   i = -0  (and only the RINEX2 file is created) 
 

In [ ]:
rnxfile=crxfile.replace(".crx",".rnx")
#!./cnvrnx3-rnx2-v3 $rnxfile +0 m 
!./cnvrnx3-rnx2-v3 $rnxfile -0 m 
rnx2file=rnxfile.replace(".rnx",".rnx.gps+glo.rnx2")

 
 Program "CNVRNX3-RNX2" V3.1.3
 for converting RINEX 3 to RINEX 2
 Copyright (C) 2012-2014 Oscar L. Colombo.
 
 Processing input file RJNI00BRA_R_20212360000_01D_15S_MO.rnx
 
 Output rinex 2.11 file: RJNI00BRA_R_20212360000_01D_15S_MO.rnx.gps+glo.rnx has been created.
 
 JOB HAS ENDED
 


## Printing the RINEX 2 converted file


In [ ]:
!head $rnx2file -n 100

     2.11           OBSERVATION DATA    M (MIXED)           RINEX VERSION / TYPE
     3.04           RINEX VERSION OF ORIGINAL FILE          COMMENT             
NetR9 5.50          Receiver Operator   20210824 000000 UTC PGM / RUN BY / DATE 
gfzrnx-1.12-7747    FILE MERGE          20210825 052040 UTC COMMENT             
RJNI                                                        MARKER NAME         
48104M001                                                   MARKER NUMBER       
RBMC                IBGE/CGED                               OBSERVER / AGENCY   
5750R51585          TRIMBLE NETR9       5.50                REC # / TYPE / VERS 
1551129599          TRM115000.00    NONE                    ANT # / TYPE        
     1     1                                                WAVELENGTH FACT L1/2
        0.0080        0.0000        0.0000                  ANTENNA: DELTA H/E/N
  4289663.4011 -4018945.7498 -2467135.8382                  APPROX POSITION XYZ 
+C2: No C2 in RINEX 3 copied

# Downloading TEQC

In [ ]:
!wget https://www.unavco.org/software/data-processing/teqc/development/teqc_CentOSLx86_64s.zip -O teqc.zip
!unzip teqc.zip 

--2021-11-01 19:43:33--  https://www.unavco.org/software/data-processing/teqc/development/teqc_CentOSLx86_64s.zip
Resolving www.unavco.org (www.unavco.org)... 69.44.86.106
Connecting to www.unavco.org (www.unavco.org)|69.44.86.106|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1087404 (1.0M) [application/zip]
Saving to: ‘teqc.zip’

teqc.zip            100%[===================>]   1.04M  --.-KB/s    in 0.09s   

2021-11-01 19:43:33 (11.5 MB/s) - ‘teqc.zip’ saved [1087404/1087404]

Archive:  teqc.zip
replace teqc? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: teqc                    


# Saving the GPS only rinex 2 file to newRinex.obs
Use the "Files" tab to download the file.

In [ ]:
newFile=rnx2file.split(".")[0]+".21o"
!./teqc -E -C -R -S -O.obs L1L2L5C1P2C5S1S2S5 $rnx2file > $newFile
print("RINEX 2 file saved to {}".format(newFile))

RINEX 2 file saved to RJNI00BRA_R_20212360000_01D_15S_MO.21o


In [ ]:
#Let's fix the lack of wavelength factor
!sed '/^.*APPROX POSITION XYZ.*/a \     1     1                                                WAVELENGTH FACT L1\/2' -i $newFile 

In [ ]:
!head $newFile -n 100

     2.11           OBSERVATION DATA    M (MIXED)           RINEX VERSION / TYPE
Linux 2.6.32-573.12.1.x86_64|x86_64|gcc -static|Linux 64|=+ COMMENT
     3.04           RINEX VERSION OF ORIGINAL FILE          COMMENT
teqc  2019Feb25                         20211101 20:06:25UTCPGM / RUN BY / DATE
NetR9 5.50          Receiver Operator   20210824 000000 UTC COMMENT
gfzrnx-1.12-7747    FILE MERGE          20210825 052040 UTC COMMENT
RJNI                                                        MARKER NAME
48104M001                                                   MARKER NUMBER
RBMC                IBGE/CGED                               OBSERVER / AGENCY
5750R51585          TRIMBLE NETR9       5.50                REC # / TYPE / VERS
1551129599          TRM115000.00    NONE                    ANT # / TYPE
     1     1                                                WAVELENGTH FACT L1/2
        0.0080        0.0000        0.0000                  ANTENNA: DELTA H/E/N
  4289663.4011 -4018945.7498